In [384]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re
from io import StringIO

# 1.0- Data Scraping

First we must pull the data from baskeball-reference.com

In [470]:
lebron = 'https://www.basketball-reference.com/players/j/jamesle01.html'
rose = 'https://www.basketball-reference.com/players/r/rosede01.html'
mj = 'https://www.basketball-reference.com/players/j/jordami01.html'
z = 'https://www.basketball-reference.com/players/z/zipsepa01.html'
players = [lebron,rose,mj,z]

In [471]:
chromedriver = "/home/michael/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [589]:
driver = webdriver.Chrome(chromedriver)
driver.get(players[2])
time.sleep(3)
expand = driver.find_element_by_xpath('//button[@id="meta_more_button"]')
time.sleep(1)
expand.click()

## 1.1 Get the Player Info

In [524]:
name = driver.find_element_by_xpath('//h1[@itemprop="name"]').text
name

'Michael Jordan'

In [539]:
height = driver.find_element_by_xpath('//span[@itemprop="height"]').text
height = height.split('-')
height = int(height[0])+(int(height[1])/12)
height

6.5

In [540]:
weight = driver.find_element_by_xpath('//span[@itemprop="weight"]').text
weight = int(weight[:-2])
weight

195

In [527]:
info = driver.find_element_by_xpath('//div[@id="info"]').text
info = info.split('\n')
info #10 Recruit 11 Draft 

['Michael Jordan',
 'Michael Jeffrey Jordan',
 '(Mike, Air Jordan, M.J., His Airness, Money, Black Cat, G.O.A.T., Superman)',
 'Position: Small Forward and Shooting Guard ▪ Shoots: Right',
 '6-6, 195lb (198cm, 88kg)',
 'Born: February 17, 1963 (Age: 54-342d) in Brooklyn, New York us',
 'College: University of North Carolina',
 'High School: Emsley A. Laney in Wilmington, North Carolina',
 'Draft: Chicago Bulls, 1st round (3rd pick, 3rd overall), 1984 NBA Draft',
 'NBA Debut: October 26, 1984',
 'Hall of Fame: Inducted as Player in 2009 (Full List)',
 'Hall of Fame',
 '14x All Star',
 '10x NBA Scoring Champ',
 '6x NBA Champ',
 '11x All-NBA',
 '9x All-Defensive',
 '1984-85 All-Rookie',
 '3x AS MVP',
 '1987-88 Def. POY',
 '6x Finals MVP',
 '5x MVP',
 '1984-85 ROY',
 '23',
 '12',
 '45',
 '23',
 'Support us without the ads? Go Ad-Free.',
 'SUMMARY',
 'Career',
 'G',
 '1072',
 'PTS',
 '30.1',
 'TRB',
 '6.2',
 'AST',
 '5.3',
 'FG%',
 '49.7',
 'FG3%',
 '32.7',
 'FT%',
 '83.5',
 'eFG%',
 '50.9'

In [528]:
draft_regex = re.compile('Draft:')
draftstr = [x  for x in info if draft_regex.search(x)]
draftstr = draftstr[0].split(',')
draftstr

['Draft: Chicago Bulls',
 ' 1st round (3rd pick',
 ' 3rd overall)',
 ' 1984 NBA Draft']

In [529]:
draftnbr = draftstr[2].strip().split(' ')
draftnbr = draftnbr[0]
draftnbr = int(draftnbr[:-2])
draftnbr

3

In [530]:
draftyr = draftstr[3].strip().split(' ')
draftyr = int(draftyr[0])
draftyr

1984

In [548]:
try:
    Trecruit_regex = re.compile('Recruiting')
    recruitstr = [x  for x in info if recruit_regex.search(x)]    
    recruitstr = recruitstr[0].split(' ')
    recruitnbr = recruitstr[-1]
    recruitnbr = int(recruitnbr[1:-1])
except:
    recruitnbr = 0
    
recruitnbr

0

In [555]:
dfscrapeinfo = pd.DataFrame(index=[0])
dfscrapeinfo['player'] = name
dfscrapeinfo['height'] = height
dfscrapeinfo['weight'] = weight
dfscrapeinfo['draftnbr']  = draftnbr
dfscrapeinfo['draftyr'] = draftyr
dfscrapeinfo['recruitnbr'] = recruitnbr
dfscrapeinfo

,player,height,weight,draftnbr,draftyr,recruitnbr
0,Michael Jordan,6.5,195,3,1984,0


## 1.2 Get the VORP Yearly Log

In [590]:
advstatsxpath = driver.find_element_by_xpath('//table[@id="advanced"]').get_attribute('outerHTML')
advstatsxpath

'<table class="row_summable sortable stats_table now_sortable sliding_cols" id="advanced" data-cols-to-freeze="1"><caption>Advanced Table</caption>\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>      \n      <tr>\n         <th aria-label="If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables." data-stat="season" scope="col" class=" poptip sort_default_asc center" data-tip="If listed as single number, the year the season ended.<br>★ - Indicates All-Star for league.<br>Only on regular season tables.">Season</th>\n         <th aria-label="Age of Player at the start of February 1st of that season." data-stat="age" scope="col" class=" poptip sort_default_asc center" data-tip="Age of Player at the start of February 1st of that season.">Age</th>\n         <th aria-label="Team" data-stat="team_id"

In [610]:
dfadvstats = pd.read_html(advstatsxpath,header=0)
dfadvstats = dfadvstats[0]
dfadvstats = dfadvstats[:-1].dropna(axis=1, how='all')
dfadvstats['player'] = name
dfadvstats

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player
0,1984-85,21.0,CHI,NBA,SG,82,3144,25.8,0.592,0.032,...,29.8,10.3,3.7,14.0,0.213,6.8,1.4,8.2,8.1,Michael Jordan
1,1985-86,22.0,CHI,NBA,SG,18,451,27.5,0.533,0.055,...,38.6,1.0,0.5,1.5,0.160,5.1,-0.5,4.7,0.8,Michael Jordan
2,1986-87,23.0,CHI,NBA,SG,82,3281,29.8,0.562,0.029,...,38.3,11.9,5.0,16.9,0.247,8.0,0.6,8.6,8.8,Michael Jordan
3,1987-88,24.0,CHI,NBA,SG,82,3311,31.7,0.603,0.027,...,34.1,15.2,6.1,21.2,0.308,9.8,2.3,12.2,11.8,Michael Jordan
4,1988-89,25.0,CHI,NBA,SG,81,3255,31.1,0.614,0.055,...,32.1,14.6,5.2,19.8,0.292,9.8,2.7,12.6,12.0,Michael Jordan
5,1989-90,26.0,CHI,NBA,SG,82,3197,31.2,0.606,0.125,...,33.7,14.7,4.3,19.0,0.285,9.7,0.8,10.6,10.1,Michael Jordan
6,1990-91,27.0,CHI,NBA,SG,82,3034,31.6,0.605,0.051,...,32.9,14.9,5.4,20.3,0.321,8.9,1.8,10.8,9.8,Michael Jordan
7,1991-92,28.0,CHI,NBA,SG,80,3102,27.7,0.579,0.055,...,31.7,12.1,5.6,17.7,0.274,6.9,1.7,8.6,8.3,Michael Jordan
8,1992-93,29.0,CHI,NBA,SG,78,3067,29.7,0.564,0.115,...,34.7,12.0,5.2,17.2,0.270,8.3,1.2,9.5,8.9,Michael Jordan
9,1994-95,31.0,CHI,NBA,SG,17,668,22.1,0.493,0.079,...,33.2,1.2,1.1,2.3,0.167,2.0,0.7,2.7,0.8,Michael Jordan


## 1.3 College Stats

In [586]:
collegexpath = driver.find_element_by_xpath('//table[@id="all_college_stats"]').get_attribute('outerHTML')

In [587]:
collegedf = pd.read_html(collegexpath,header=0)
collegedf = collegedf[0].dropna(axis=0, thresh=4)
collegedf.columns = collegedf.iloc[0]
collegedf = collegedf.reindex(collegedf.index.drop(0))
collegedf['name'] = name
collegedf

,Season,Age,College,G,MP,FG,FGA,3P,3PA,FT,...,PF,PTS,FG%,3P%,FT%,MP,PTS,TRB,AST,name
1,1981-82,18,UNC,34,NaN,191,358,NaN,NaN,78,...,91,460,.534,NaN,.722,NaN,13.5,4.4,1.8,Michael Jordan
2,1982-83,19,UNC,36,NaN,282,527,34,76,123,...,110,721,.535,.447,.737,NaN,20.0,5.5,1.6,Michael Jordan
3,1983-84,20,UNC,31,NaN,247,448,NaN,NaN,113,...,70,607,.551,NaN,.779,NaN,19.6,5.3,2.1,Michael Jordan
4,Career,NaN,NaN,101,NaN,720,1333,34,76,314,...,271,1788,.540,.447,.748,NaN,17.7,5.0,1.8,Michael Jordan
